# 1. 라이브러리 임포트

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
import cv2
import dlib

# 2. 이미지에 스티커 추가하기
- 딥러닝 모델을 통한 얼굴 영역 검출 + dlib를 사용하여 얼굴 랜드마크 검출
- 얼굴 좌표 에러 해결

In [ ]:
def img2sticker(img_orig, img_sticker, landmark_predictor):
    
    (h, w) = img_orig.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(img_orig, (300, 300)), 1.0,
    (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()
    
    rects = []
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence < 0.9:
            continue

        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        rect = dlib.rectangle(startX, startY, endX, endY)
        rects.append(rect)

    list_landmarks = []
    for rect in rects:
        points = landmark_predictor(img_orig, rect)
        list_points = list(map(lambda p: (p.x, p.y), points.parts()))
        list_landmarks.append(list_points)
    
    if not list_landmarks:
        return img_orig

    for rect, landmark in zip(rects, list_landmarks):
        x = landmark[30][0] # nose
        y = landmark[30][1] - rect.width()//2
        w = rect.width()
        h = rect.width()
        print(x, y, w, h)
        break
    
    # sticker
    img_sticker = cv2.resize(img_sticker, (w,h), interpolation=cv2.INTER_NEAREST)
    refined_x = x - w // 2
    refined_y = y - h
    
    if refined_y < 0:
        img_sticker = img_sticker[-refined_y:]
        refined_y = 0
    if refined_x < 0:
        img_sticker = img_sticker[:, -refined_x:]
        refined_x = 0
    elif refined_x + img_sticker.shape[1] >= img_orig.shape[1]:
        img_sticker = img_sticker[:, :-(img_sticker.shape[1]+refined_x-img_orig.shape[1])]

    img_bgr = img_orig.copy()
    sticker_area = img_bgr[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]]

    img_bgr[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]] = \
        cv2.addWeighted(sticker_area, 1.0, img_sticker, 0.7, 0)

    return img_bgr

# 3. 카메라 스트리밍을 통한 스티커앱

In [ ]:
net = cv2.dnn.readNetFromCaffe("deploy.prototxt.txt", "res10_300x300_ssd_iter_140000.caffemodel")
landmark_predictor = dlib.shape_predictor('./models/shape_predictor_68_face_landmarks.dat')

def main():
    cv2.namedWindow('show', 0)
    cv2.resizeWindow('show', 640, 360)

    vc = cv2.VideoCapture(0)
    img_sticker = cv2.imread('./images/king.png')

    vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
    while True:
        ret, img = vc.read()
        if ret == False:
            break
        start = cv2.getTickCount()
        img = cv2.flip(img, 1)  # 보통 웹캠은 좌우 반전
        
        img_result = img2sticker(img, img_sticker.copy(), landmark_predictor)   

        time = (cv2.getTickCount() - start) / cv2.getTickFrequency() * 1000
        print ('[INFO] time: %.2fms'%time)
        cv2.imshow('show', img_result)
        key = cv2.waitKey(1)
        if key == 27:
            break

if __name__ == '__main__':
    main()

# 4. 정리

- 기존의 dlib 라이브러리를 사용하여 얼굴 검출을 할 시 이미지 피라미드 개수에 따라서 계산 속도의 차이가 컸으며, 다양한 거리에서의 얼굴을 검출하기 위해서는 계산 시간이 훨씬 오래걸렸습니다.

- caffe 프레임워크로 개발된 resnet 모델을 사용하여, 기존에 dlib로 얼굴 영역 검출 부분을 대체시킴으로서 다양한 거리와 방향을 보는 얼굴들을 검출할 수 있게 되었으며 더 빠르게 검출해 낼 수가 있었습니다.

- 얼굴이 좌측을 넘어 갈 때 x 좌표계 값이 음수가 되는 문제를 해결하여 기존의 스티커 앱에서 발생하던 오류를 제거 할 수있었습니다.